In [ ]:
import boto3
from attr import dataclass
from pyspark.sql import SparkSession
from pyspark.sql import functions as F  # noqa
from pyspark.sql import types as T  # noqa

# 1. 자격 증명 로드 (Boto3)
source_credential = boto3.Session(profile_name="prod").get_credentials()
target_credential = boto3.Session(profile_name="qa").get_credentials()


@dataclass(frozen=True)
class Catalog:
    name: str
    warehouse: str


SOURCE = Catalog(name="source", warehouse="s3a://hunet-di-data-lake-prod/iceberg")
TARGET = Catalog(name="target", warehouse="s3a://hunet-di-data-lake-qa/iceberg")

# 2. Spark 세션 생성 (카탈로그 구조만 정의)
spark = (
    SparkSession.builder.appName("IcebergMigrationFinal")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config(f"spark.sql.catalog.{SOURCE.name}", "org.apache.iceberg.spark.SparkCatalog")
    .config(f"spark.sql.catalog.{SOURCE.name}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
    .config(f"spark.sql.catalog.{SOURCE.name}.warehouse", SOURCE.warehouse)
    .config(f"spark.sql.catalog.{SOURCE.name}.s3.path-style-access", True)
    .config(f"spark.sql.catalog.{TARGET.name}", "org.apache.iceberg.spark.SparkCatalog")
    .config(f"spark.sql.catalog.{TARGET.name}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
    .config(f"spark.sql.catalog.{TARGET.name}.warehouse", TARGET.warehouse)
    .config(f"spark.sql.catalog.{TARGET.name}.s3.path-style-access", True)
    .config("spark.driver.memory", "8g")
    .config("spark.sql.caseSensitive", True)
    .config("spark.sql.session.timeZone", "UTC")
    .getOrCreate()
)

# Java 시스템 프로퍼티 제어를 위한 JVM 게이트웨이
jvm = spark.sparkContext._gateway.jvm


def set_java_aws_credentials(cred):
    """Java SDK v2가 사용하는 시스템 프로퍼티를 강제로 덮어씀"""
    jvm.java.lang.System.setProperty("aws.accessKeyId", cred.access_key)
    jvm.java.lang.System.setProperty("aws.secretAccessKey", cred.secret_key)
    if cred.token:
        jvm.java.lang.System.setProperty("aws.sessionToken", cred.token)
    else:
        jvm.java.lang.System.clearProperty("aws.sessionToken")


def migration(schema, table):
    try:
        # --- STEP 1: Account A 자격 증명 주입 및 데이터 읽기 ---
        print("INFO: Accessing Account A (Prod)")
        set_java_aws_credentials(source_credential)

        # 중요: collect()를 사용하여 데이터를 Python 메모리로 완전히 가져옵니다.
        # Spark의 Lazy Evaluation 때문에 나중에 쓰려고 하면 인증 정보가 꼬일 수 있습니다.
        source_df = spark.read.table(f"{SOURCE.name}.{schema}.{table}")

        print("INFO: Materializing data from Account A...")
        # 데이터가 너무 크면 .collect() 대신 temporary path에 저장을 고려해야 합니다.
        # rows = source_df.collect()
        # local_df = spark.createDataFrame(rows, schema=source_df.schema)

        source_df.write.mode("overwrite").format("parquet").save("/Users/kimyj/tmp_spark")
        local_df = spark.read.format("parquet").load("/Users/kimyj/tmp_spark")

        # --- STEP 2: Account B 자격 증명 주입 및 데이터 쓰기 ---
        print("INFO: Switching to Account B (QA)")
        set_java_aws_credentials(target_credential)

        # 이제 환경 변수가 아닌 Java System Property가 B 계정이므로
        # Glue Catalog B와 S3 B에 정상적으로 접근합니다.
        # spark.sql(f"CREATE DATABASE IF NOT EXISTS {TARGET.name}.{schema}")
        (local_df.writeTo(f"{TARGET.name}.{schema}.{table}").tableProperty("format-version", "2").createOrReplace())
        print("INFO: Data migration completed successfully.")

    except Exception as e:
        print(f"ERROR: Migration failed during JVM execution: {str(e)}")
        raise e


print("INFO: Accessing Account A (Prod)")
set_java_aws_credentials(source_credential)
spark.catalog.setCurrentCatalog(SOURCE.name)
tables = spark.catalog.listTables("")
for i, table in enumerate(tables, start=1):
    print(f"{table.namespace[0]}.{table.name}")

In [ ]:
tables = []

try:
    for i, table in enumerate(tables, start=1):
        schema, table = table.split(".")
        print(f"INFO: Migrating table {i}...{schema}.{table}")
        migration(schema=schema, table=table)
finally:
    # 작업 종료 후 보안을 위해 시스템 프로퍼티 초기화
    jvm.java.lang.System.clearProperty("aws.accessKeyId")
    jvm.java.lang.System.clearProperty("aws.secretAccessKey")
    jvm.java.lang.System.clearProperty("aws.sessionToken")